In [32]:
import numpy as np
import pandas as pd
import os
import glob
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso
import itertools

In [2]:
cwd = os.getcwd()
cwd

'C:\\Users\\roshi\\Downloads\\furnace_data\\furnace_data'

In [216]:
df=pd.read_csv('furnace_cycle-001.csv')
df.head()

,Line,DateTime,Door_Status,Metal_Temp,Burner_Input,Current_Roof_Temp,Roof_Temp_SetPoint
0,3397,21:21:00,0,NaN,0.0,808.205078,644.425354
1,3398,21:22:00,0,NaN,0.0,718.014282,644.425354
2,3399,21:23:00,0,NaN,0.0,657.406799,644.425354
3,3400,21:24:00,0,NaN,0.0,617.707275,644.425354
4,3401,21:25:00,0,NaN,0.0,589.156677,644.425354


### Dataset: furnace_cylce_merged

In [ ]:
for i in range(1,10):
    a='furnace_cycle-00'+str(i)+'.csv'
    df=pd.read_csv(a)
    df['Mins'] = np.arange(df.shape[0])
    df.to_csv('C:\\Users\\roshi\\Downloads\\furnace_data\\furnace_cycle-00'+str(i)+'.csv',index=False)
    
for i in range(10,38):
    a='furnace_cycle-0'+str(i)+'.csv'
    df=pd.read_csv(a)
    df['Mins'] = np.arange(df.shape[0])
    df.to_csv('C:\\Users\\roshi\\Downloads\\furnace_data\\furnace_cycle-00'+str(i)+'.csv',index=False)

In [ ]:
files = os.path.join("C:\\Users\\roshi\\Downloads\\furnace_data\\", "furnace_cycle-*.csv")
files = glob.glob(files)
df = pd.concat(map(pd.read_csv, files), ignore_index=True)
df.to_csv('C:\\Users\\roshi\\Downloads\\furnace_data\\furnace_cylce_merged.csv',index=False)

In [ ]:
for i in range(1,10):
    a='furnace_cycle-00'+str(i)+'.csv'
    df=pd.read_csv(a)
    df['Mins'] = np.arange(df.shape[0])
    df['Cycle']=i
    df=df[['DateTime', 'Mins', 'Cycle','Door_Status', 'Metal_Temp','Current_Roof_Temp']]
    df['roof_temperature_1']=0
    df['roof_temperature_2']=0
    for index in range(1,df.shape[0]):
        a=index-1
        df['roof_temperature_1'][index]=df['Current_Roof_Temp'][a]
        df['roof_temperature_2'][index]=df['roof_temperature_1'][a]
        df.to_csv('C:\\Users\\roshi\\Downloads\\furnace_data\\furnace_cycle-00'+str(i)+'.csv',index=False)
    
for i in range(10,38):
    a='furnace_cycle-0'+str(i)+'.csv'
    df=pd.read_csv(a)
    df['Mins'] = np.arange(df.shape[0])
    df['Cycle']=i
    df=df[['DateTime', 'Mins', 'Cycle','Door_Status', 'Metal_Temp','Current_Roof_Temp']]
    df['roof_temperature_1']=0
    df['roof_temperature_2']=0
    for index in range(1,df.shape[0]):
        a=index-1
        df['roof_temperature_1'][index]=df['Current_Roof_Temp'][a]
        df['roof_temperature_2'][index]=df['roof_temperature_1'][a]
        df.to_csv('C:\\Users\\roshi\\Downloads\\furnace_data\\furnace_cycle-00'+str(i)+'.csv',index=False)

In [261]:
df=pd.read_csv('C:\\Users\\roshi\\Downloads\\furnace_data\\furnace_cycle-001.csv')
df.head()

,DateTime,Mins,Cycle,Door_Status,Metal_Temp,Current_Roof_Temp,roof_temperature_1,roof_temperature_2
0,21:21:00,0,1,0,NaN,808.205078,0.000000,0.000000
1,21:22:00,1,1,0,NaN,718.014282,808.205078,0.000000
2,21:23:00,2,1,0,NaN,657.406799,718.014282,808.205078
3,21:24:00,3,1,0,NaN,617.707275,657.406799,718.014282
4,21:25:00,4,1,0,NaN,589.156677,617.707275,657.406799


### Dataset: furnace_cylce_merged

In [262]:
files = os.path.join("C:\\Users\\roshi\\Downloads\\furnace_data\\", "furnace_cycle-*.csv")
files = glob.glob(files)
df = pd.concat(map(pd.read_csv, files), ignore_index=True)

In [263]:
df=df.drop(index=[0,1]).reset_index(drop=True)
df.head()

,DateTime,Mins,Cycle,Door_Status,Metal_Temp,Current_Roof_Temp,roof_temperature_1,roof_temperature_2
0,21:23:00,2,1,0,NaN,657.406799,718.014282,808.205078
1,21:24:00,3,1,0,NaN,617.707275,657.406799,718.014282
2,21:25:00,4,1,0,NaN,589.156677,617.707275,657.406799
3,21:26:00,5,1,0,NaN,567.812683,589.156677,617.707275
4,21:27:00,6,1,0,NaN,550.760437,567.812683,589.156677


In [264]:
#df=df.drop(index=[0,1]).reset_index(drop=True)
#df.head()
df=df.loc[df['Metal_Temp'].notnull()]
df=df.loc[df['Door_Status']==1]
df=df.reset_index().drop(columns=['index'])

In [268]:
df=df[['Mins', 'Metal_Temp','Current_Roof_Temp', 'roof_temperature_1', 'roof_temperature_2']]

In [269]:
df.head()

,Mins,Metal_Temp,Current_Roof_Temp,roof_temperature_1,roof_temperature_2
0,385,706.0,1118.423218,1118.415527,1112.462524
1,386,705.0,1118.556641,1118.423218,1118.415527
2,387,705.0,1117.272461,1118.556641,1118.423218
3,388,705.0,1111.591064,1117.272461,1118.556641
4,389,705.0,1135.606567,1111.591064,1117.272461


In [270]:
df.to_csv('C:\\Users\\roshi\\Downloads\\furnace_data\\furnace_cylce_merged_final.csv',index=False)

df=pd.read_csv('C:\\Users\\roshi\\Downloads\\furnace_data\\furnace_cylce_merged_final.csv')


In [276]:
df.head()

,Mins,Metal_Temp,Current_Roof_Temp,roof_temperature_1,roof_temperature_2
0,385,706.0,1118.423218,1118.415527,1112.462524
1,386,705.0,1118.556641,1118.423218,1118.415527
2,387,705.0,1117.272461,1118.556641,1118.423218
3,388,705.0,1111.591064,1117.272461,1118.556641
4,389,705.0,1135.606567,1111.591064,1117.272461


### Dataset : BigData_imputed

In [ ]:
data= pd.read_csv('BigData_2018_2019.csv')
def ranges(i):
    for a, b in itertools.groupby(enumerate(i), lambda pair: pair[1] - pair[0]):
        b = list(b)
        yield b[0][1], b[-1][1]
null_index= list(ranges(list(data[data['Metal_Temp'].isnull()].index)))
data['Impute']=0
for i in null_index:
    range_1= int(data.iloc[i[1]+1]['Metal_Temp'])
    range_2= int(data.iloc[i[0]-1]['Metal_Temp'])   
    if range_1>range_2:
        impute_list= np.random.randint(range_2, range_1, i[1]-i[0]+1)
    elif range_1==range_2:
        impute_list= np.random.randint(range_1-1, range_2, i[1]-i[0]+1)
    else:
        impute_list= np.random.randint(range_1, range_2, i[1]-i[0]+1)
    k=0
    for j in range(i[0],i[1]+1):
        data['Metal_Temp'][j]= impute_list[k]
        data['Impute'][j]=1
        k+=1
data= data[data['Door_Status']==1][['DateTime', 'Metal_Temp', 'Current_Roof_Temp', 'Impute']]
data.to_csv('BigData_imputed.csv')